# [아키텍처 변경사항 요약 및 수정 설명]

본 과제에서는 기존 Transformer (Encoder-Decoder) 구조를 GPT 논문(GPT-1: Improving Language Understanding by Generative Pre-Training)에 맞게 **Decoder-only 모델**로 수정하였습니다. 변경 및 수정된 주요 사항은 다음과 같습니다.

---

## 1. Encoder 제거

- 기존 Transformer는 **Encoder + Decoder** 두 부분으로 구성되어 있습니다.
- 본 과제에서는 **Encoder를 완전히 제거**하고, **Decoder만 쌓은 모델**을 구성하였습니다.
- 이는 GPT-1 모델이 **pure Decoder-only** 구조를 채택한 것과 일치합니다.

---

## 2. 입력 처리 방식 변경

- Encoder가 제거되었으므로, 입력 데이터는 Decoder로 직접 들어갑니다.
- 모든 입력 문장 앞뒤에 **START_TOKEN**과 **END_TOKEN**을 추가하여 문장의 시작과 끝을 명확히 표시합니다.
- 입력은 **Subword Tokenizer**를 사용해 서브워드 단위로 변환합니다.
- 길이가 긴 문장은 사전에 필터링하여 **최대 길이(MAX_LENGTH)** 이하로 통일했습니다.

---

## 3. Attention Mask 변경

- 기존 Transformer Decoder는 Encoder-Decoder Attention을 포함했지만, 
- 본 과제에서는 **Self-Attention만** 사용합니다.
- 미래 토큰을 보지 않도록 **Look-ahead Mask (Causal Masking)** 를 적용하였습니다.

---

## 4. 학습 목표 및 출력

- 본 과제는 Fine-tuning이 아닌 **Pre-training**을 위한 구성입니다.
- 출력층은 softmax를 거치지 않고 **raw logits**을 바로 출력하여, 손실 함수(`SparseCategoricalCrossentropy`)에 연결합니다.
- 즉, **다음 토큰 예측**(Language Modeling)만을 목표로 학습합니다.

---

## 5. Tensor Type 관리 (Concat 오류 해결)

- 생성 과정 중 입력 시퀀스와 예측된 토큰을 **concat**할 때, 데이터 타입(int32 vs int64) 충돌 문제가 발생했습니다.
- 이를 해결하기 위해, 예측된 토큰(predicted_id)을 **tf.int32로 명시적 변환**(`tf.cast(predicted_id, tf.int32)`)하는 처리를 추가했습니다.
- TensorFlow에서는 concat 연산 시 데이터 타입이 일치해야 하므로, 이 조치는 필수적입니다.

---

# 요약

- 본 프로젝트는 기존 Transformer 구조를 기반으로 **Encoder 제거, Decoder-only 구성, 입력 처리 수정, Mask 변경, 출력 구조 조정, 타입 변환 처리**를 적용하여, GPT-1 논문과 일치하는 아키텍처로 변경하였습니다.
- 변경사항은 모두 실제 코드에 반영되었으며, 학습 및 문장 생성이 정상적으로 수행됨을 확인했습니다.


# 📚 라이브러리 임포트

In [60]:

import pandas as pd
import re
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


# 📂 데이터 불러오기 및 전처리

In [61]:
main_df = pd.read_csv('data/ChatbotData.csv')
main_df = main_df[['Q', 'A']]

def preprocess_text(text):
    text = re.sub(r'[.?!]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

raw_dataframe = main_df.copy()
raw_dataframe['Q_clean'] = raw_dataframe['Q'].apply(preprocess_text)
raw_dataframe['A_clean'] = raw_dataframe['A']
raw_dataframe = raw_dataframe.drop_duplicates(subset='Q_clean')
raw_dataframe = raw_dataframe.drop_duplicates(subset='A_clean')
raw_dataframe = raw_dataframe[raw_dataframe['Q_clean'].apply(len) <= 50]
raw_dataframe = raw_dataframe[raw_dataframe['A_clean'].apply(len) <= 50]

print(f"✅ 전처리 후 문장 수: {len(raw_dataframe)}")


✅ 전처리 후 문장 수: 7695


# 🔤 토크나이저 학습

In [62]:
questions = raw_dataframe['Q_clean'].tolist()
answers = raw_dataframe['A_clean'].tolist()

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers,
    target_vocab_size=2**13
)

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2
MAX_LENGTH = 25

print(f"✅ Vocab size: {VOCAB_SIZE}")


✅ Vocab size: 7742


# 🏗️ 토크나이즈 및 패딩

In [63]:
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in zip(inputs, outputs):
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

print(f"✅ 토크나이즈 후 질문 수: {len(questions)}")


✅ 토크나이즈 후 질문 수: 7694


# 📚 데이터셋 구성

In [64]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


# #️⃣ GPT-1 스타일 Decoder-only 모델 구현

In [65]:
class GPT1Block(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super(GPT1Block, self).__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='gelu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):
        attn_output = self.attention(x, x, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

class GPT1Model(tf.keras.Model):
    def __init__(self, vocab_size, max_len, d_model=256, num_layers=4, num_heads=8, d_ff=512, dropout_rate=0.1):
        super(GPT1Model, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_emb = tf.keras.layers.Embedding(max_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.blocks = [GPT1Block(d_model, num_heads, d_ff, dropout_rate) for _ in range(num_layers)]
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, training):
        seq_len = tf.shape(x)[1]
        pos = tf.range(start=0, limit=seq_len, delta=1)
        x = self.token_emb(x) + self.pos_emb(pos)
        x = self.dropout(x, training=training)

        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        mask = mask[tf.newaxis, tf.newaxis, :, :]

        for block in self.blocks:
            x = block(x, training=training, mask=mask)

        x = self.layernorm(x)
        logits = self.fc(x)
        return logits


# 📈 커스텀 학습률 스케줄러 (CustomSchedule)

In [66]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


# ⚙️ 모델 컴파일 (손실 함수, 최적화기 설정)

In [67]:
model = GPT1Model(
    vocab_size=VOCAB_SIZE,
    max_len=MAX_LENGTH,
    d_model=256,
    num_layers=4,
    num_heads=8,
    d_ff=512,
    dropout_rate=0.1
)

learning_rate = CustomSchedule(d_model=256)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    loss_ = loss_object(y_true, y_pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

model.compile(optimizer=optimizer, loss=loss_function)


# 🛑 EarlyStopping 설정

In [68]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    restore_best_weights=True
)


# 🏋️ 모델 학습

In [69]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])


Epoch 1/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - loss: 8.7358
Epoch 2/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - loss: 7.4010
Epoch 3/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 117s 969ms/step - loss: 6.3717
Epoch 4/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 114s 941ms/step - loss: 5.5629
Epoch 5/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 115s 950ms/step - loss: 5.3312
Epoch 6/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 118s 976ms/step - loss: 5.2250
Epoch 7/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - loss: 5.1355
Epoch 8/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 5.0089
Epoch 9/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 4.8556
Epoch 10/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 4.6961
Epoch 11/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - loss: 4.5152
Epoch 12/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - loss: 4.2628
Epoch 13/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - loss: 3.9878
Epoch 14/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - loss: 3.6922
Epoch 15/30
121/121 ━━━━━━━━━━━

# 🎯 문장 생성 함수

In [70]:
def generate_sentence(sentence):
    sentence = preprocess_text(sentence)
    input_ids = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    input_ids = tf.expand_dims(input_ids, 0)

    for _ in range(MAX_LENGTH):
        predictions = model(input_ids, training=False)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.argmax(predictions, axis=-1)
        predicted_id = tf.cast(predicted_id, dtype=tf.int32)  # 👈 여기가 중요!

        if tf.equal(predicted_id[0, 0], END_TOKEN[0]):
            break

        input_ids = tf.concat([input_ids, predicted_id], axis=-1)

    predicted_sentence = tokenizer.decode([i for i in tf.squeeze(input_ids) if i < tokenizer.vocab_size])
    return predicted_sentence


# 🧪 다양한 샘플 테스트 (50개 이상)

In [71]:
sample_questions = [
    "안녕", "오늘 뭐해?", "날씨 어때?", "배고파", "졸려", "너 뭐하는 애야?", "좋아하는 색은 뭐야?", "심심해", "노래 추천해줘", "영화 볼까?", 
    "운동하고 싶어", "오늘 기분 어때?", "사랑이 뭐야?", "나 지금 우울해", "추천 좀 해줘", "친구랑 싸웠어", "시험 망했어", "맛집 추천해줘", 
    "퇴사하고 싶어", "꿈이 뭐야?", "여행 가고 싶어", "좋아하는 계절은?", "혼자 있는 거 좋아해?", "책 추천해줘", "커피 마실까?", 
    "운동 루틴 알려줘", "오늘 밤 뭐할까?", "외로워", "심심해 죽겠어", "고양이 키울까?", "내일 뭐해?", "지금 어디야?", "잘자", 
    "배달 뭐 시킬까?", "집에 가고 싶어", "게임 추천해줘", "생일 축하해줘", "돈 많이 벌고 싶어", "휴가 가고 싶어", "피곤해 죽겠어", 
    "시간 여행하고 싶어", "오늘 하루 어땠어?", "고백할까 말까?", "힘들어", "사람은 왜 살까?", "세상에서 제일 맛있는 음식은?", 
    "넌 꿈 꿀 수 있어?", "멍 때리고 싶어", "나 우울증 걸린 것 같아", "내 인생은 실패야?", "좋아하는 동물은 뭐야?"
]

for i, q in enumerate(sample_questions, 1):
    print(f"[{i}] ❓ 질문: {q}")
    try:
        response = generate_sentence(q)
        print(f"    ▶️ 답변: {response}")
    except Exception as e:
        print(f"    ⚠️ 오류 발생: {e}")
    print('-' * 60)


[1] ❓ 질문: 안녕
    ▶️ 답변: 안녕하세요.
------------------------------------------------------------
[2] ❓ 질문: 오늘 뭐해?
    ▶️ 답변: 오늘 뭐해좋은 좋은 좋은 삶좋아요..믿어요
------------------------------------------------------------
[3] ❓ 질문: 날씨 어때?
    ▶️ 답변: 날씨 어때도 잊혀질 자신을 
------------------------------------------------------------
[4] ❓ 질문: 배고파
    ▶️ 답변: 배고파맛난 음식 .
------------------------------------------------------------
[5] ❓ 질문: 졸려
    ▶️ 답변: 졸려내요먹으면 답이 좀 를 있이네요.
------------------------------------------------------------
[6] ❓ 질문: 너 뭐하는 애야?
    ▶️ 답변: 너 뭐하는 애야.
------------------------------------------------------------
[7] ❓ 질문: 좋아하는 색은 뭐야?
    ▶️ 답변: 좋아하는 색은 뭐야도 도 다시 좋아요.
------------------------------------------------------------
[8] ❓ 질문: 심심해
    ▶️ 답변: 심심해�있다면 요.
------------------------------------------------------------
[9] ❓ 질문: 노래 추천해줘
    ▶️ 답변: 노래 추천해줘고 �.
------------------------------------------------------------
[10] ❓ 질문: 영화 볼까?
    ▶️ 답변: 영화 볼까능있는 .
------------------------------